In [0]:
!pip install geopandas

In [0]:
import numpy as np

import pandas as pd
import geopandas as gpd

import psycopg2
import osgeo.ogr
import matplotlib.pyplot as plt

import datetime as dt

from shapely.geometry import Point
from shapely.ops import nearest_points

import folium

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = "/content/drive/My Drive/PSDA/" 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
connection = psycopg2.connect(host="193.196.37.97", database="postgres",user="postgres", password="SDAPraktikum2020")
cursor = connection.cursor()  

In [0]:
#stations_data = gpd.read_postgis('SELECT * FROM air_temperature_stations', connection, geom_col='geom_station')
stations = gpd.read_postgis('SELECT stations_id, stationsname, bundesland, geom_station FROM air_temperature_stations', connection, geom_col='geom_station', index_col='stations_id')
stations = stations[stations['bundesland'] == 'Baden-Württemberg']

In [0]:
waterways = gpd.read_file(base_dir + 'Data/waterways.dbf')
rivers = waterways[waterways['type'] == 'river']

points = []

multiline_rivers = rivers.geometry.unary_union

for index, row in stations.iterrows():
  point = row.geom_station  
  queried_geom, nearest_geom = nearest_points(point, multiline_rivers)
  points.append(nearest_geom)

closest_rivers = gpd.GeoDataFrame(geometry = points)

In [0]:
distances = []
count =  0 
for index, row in stations.iterrows():
  dist = closest_rivers.iloc[count].geometry.distance(stations.iloc[count].geom_station)
  distances.append(dist)
  count = count + 1
  
stations['water_distance'] = distances
stations

,stationsname,bundesland,geom_station,water_distance
stations_id,,,,
257,Baden-Baden-Geroldsau,Baden-Württemberg,POINT (8.24570 48.72700),0.002100
259,Müllheim,Baden-Württemberg,POINT (7.63870 47.80640),0.001318
268,Baiersbronn-Obertal,Baden-Württemberg,POINT (8.27270 48.52930),0.010746
474,Warthausen-Birkenhard,Baden-Württemberg,POINT (9.76390 48.12520),0.032325
755,"Buchen, Kr, Neckar-Odenwald",Baden-Württemberg,POINT (9.32130 49.51820),0.163905
...,...,...,...,...
7403,Leutkirch-Herlazhofen,Baden-Württemberg,POINT (10.03240 47.79550),0.008034
13667,Freiburg-Mitte,Baden-Württemberg,POINT (7.84500 48.00060),0.009884
13674,Waibstadt,Baden-Württemberg,POINT (8.90530 49.29430),0.049919


In [14]:
# Create map showing the rivers in Germany
m = folium.Map(location=[51.3, 10.1], zoom_start=6)  # This zooms the map to focus on germany (coods 51.3, 10.1)

choropleth = folium.Choropleth(
    geo_data = waterways[waterways['type'] == 'river'],
    geo_str = 'choropleth',
    fillColor = '#0000ff',
    color = '#0000ff',
    weight=10,
    fill_opacity=100000,
    line_opacity=100000,
    legend_name='Temperature (celsius)',
    highlight=True
).add_to(m)

# Display the map
m

Output hidden; open in https://colab.research.google.com to view.